In [26]:
import pandas as pd
import matplotlib.pylab as plt
import surprise
import numpy as np
from surprise import Reader
from surprise import Dataset

In [3]:
# Read csv
trade_df = pd.read_csv("./data/trade.csv", encoding="utf-8", parse_dates=['ISSUEDT'],
                       dtype={'HSCD': str, "QTY": float, 'BYRADDR2': str, 'SPLYADDR2': str})

In [9]:
trade_df.sort_values('ISSUEDT', ascending=False)

,HSCD,QTY,QTYUNIT,AMT,CUR,USDUNTPRICE,UNTPRICE,ITEM,BYRORGNM1,BYRPRTNUM,...,BYRADDR2,SPLYORGNM1,SPLYPRTNUM,SPLYADDR1,SPLYADDR2,ISSUEDT,TOTQTY,TOTQTYUNIT,TOTAMT,TOTAMTCUR
2492785,8532240000,300000.0,PCE,13200.00,JPY,0.00,0.00,"C-CER,CHIP",삼성전자(주),1248100998,...,P1C2,한국무라타전자(주),1208615359,서울시강남구대치동942해성1빌딩5층,NaN,2018-02-28,300000.0,PCE,1.320000e+04,JPY
1694071,1905901090,30.0,CTN,543.00,USD,NaN,18.10,오뜨 프로마즈,(주)에스케이트레이딩,2128609097,...,) 서경빌딩 303호,디에스타(Desta International Inc),1102012344,서울특별시 은평구 갈현로1길,"6-3, 202호(신사동,그린아트빌)",2018-02-28,970.0,CTN,1.496705e+04,USD
1694084,6307909000,200.0,PC,7780000.00,KRW,NaN,38900.00,ROPE COVER,팁코,6212258150,...,05호,이스턴산업,2178136032,서울특별시 노원구 화랑로 465 411,"(공릉동, 엘네스트빌 오피스텔)",2018-02-28,200.0,PC,7.780000e+06,KRW
1694083,2206002030,20.0,CT,165.60,USD,NaN,8.28,쌀막걸리,보성특산교역(주),2158121436,...,NaN,(주)국순당,1248114925,강원도 횡성군 둔내면 강변로 975,NaN,2018-02-28,20.0,CT,1.656000e+02,USD
1694082,1905901090,20.0,CTN,243.80,USD,NaN,12.19,바이오캔디(M),(주)에스케이트레이딩,2128609097,...,) 서경빌딩 303호,디에스타(Desta International Inc),1102012344,서울특별시 은평구 갈현로1길,"6-3, 202호(신사동,그린아트빌)",2018-02-28,970.0,CTN,1.496705e+04,USD
1694081,1905901090,30.0,CTN,369.30,USD,NaN,12.31,통아몬드(S),(주)에스케이트레이딩,2128609097,...,) 서경빌딩 303호,디에스타(Desta International Inc),1102012344,서울특별시 은평구 갈현로1길,"6-3, 202호(신사동,그린아트빌)",2018-02-28,970.0,CTN,1.496705e+04,USD
1694080,1905901090,20.0,CTN,327.00,USD,NaN,16.35,다이제 초코,(주)에스케이트레이딩,2128609097,...,) 서경빌딩 303호,디에스타(Desta International Inc),1102012344,서울특별시 은평구 갈현로1길,"6-3, 202호(신사동,그린아트빌)",2018-02-28,970.0,CTN,1.496705e+04,USD
1694079,1905901090,20.0,CTN,260.00,USD,NaN,13.00,다이제 오리지널,(주)에스케이트레이딩,2128609097,...,) 서경빌딩 303호,디에스타(Desta International Inc),1102012344,서울특별시 은평구 갈현로1길,"6-3, 202호(신사동,그린아트빌)",2018-02-28,970.0,CTN,1.496705e+04,USD
1694078,1905901090,20.0,CTN,354.20,USD,NaN,17.71,초코칩(M),(주)에스케이트레이딩,2128609097,...,) 서경빌딩 303호,디에스타(Desta International Inc),1102012344,서울특별시 은평구 갈현로1길,"6-3, 202호(신사동,그린아트빌)",2018-02-28,970.0,CTN,1.496705e+04,USD
1694077,1905901090,25.0,CTN,352.25,USD,NaN,14.09,예감 갈릭버터(S),(주)에스케이트레이딩,2128609097,...,) 서경빌딩 303호,디에스타(Desta International Inc),1102012344,서울특별시 은평구 갈현로1길,"6-3, 202호(신사동,그린아트빌)",2018-02-28,970.0,CTN,1.496705e+04,USD


In [10]:
trade_df = trade_df[['BYRORGNM1', 'SPLYORGNM1']]

In [11]:
df = trade_df.groupby(['BYRORGNM1', 'SPLYORGNM1']).agg('size').reset_index()

In [12]:
df.columns = ['byr', 'spl', 'cnt']

In [13]:
# Drop rows under 5 evaluations
b = df['byr'].value_counts() >= 5
b = b[b].index.tolist()
s = df['spl'].value_counts() >= 5
s = s[s].index.tolist()
df = df.loc[(df['byr'].isin(b)) & (df['spl'].isin(s))].reset_index(drop=True)

In [14]:
# Drop rows under 5 evaluations
b = df['byr'].value_counts() >= 5
b = b[b].index.tolist()
df = df.loc[df['byr'].isin(b)].reset_index(drop=True)

In [15]:
totalcnt_byr_df = df.groupby('byr').agg('sum')

In [16]:
def div_cnt(cnt: int, total_cnt: int) -> float:
    return cnt/total_cnt

In [17]:
df['div_cnt'] = df.apply(lambda x: div_cnt(x.cnt, totalcnt_byr_df.loc[x.byr].cnt), axis=1)

In [18]:
df = df.drop('cnt', axis=1)

In [19]:
df.div_cnt.max()

0.9694807768529529

In [20]:
df.div_cnt.min()

1.687564338390401e-05

In [21]:
reader = Reader(rating_scale=(0.0,1.0))
data = Dataset.load_from_df(df, reader)

In [22]:
from surprise.model_selection import cross_validate
from surprise import SVD
from surprise import SVDpp
from surprise import SlopeOne
from surprise import NMF
from surprise import NormalPredictor
from surprise import KNNBaseline
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import BaselineOnly
from surprise import CoClustering

benchmark = []
# Iterate over all algorithms
for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse') 

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...


,fit_time,test_rmse,test_time
Algorithm,,,
BaselineOnly,0.006783,0.107336,0.008260
KNNBaseline,0.016123,0.114993,0.036232
SVDpp,1.596674,0.117110,0.066581
KNNBasic,0.009710,0.119769,0.029237
SlopeOne,0.051746,0.120795,0.037096
KNNWithMeans,0.018146,0.123324,0.031188
NMF,0.392615,0.125772,0.011338
KNNWithZScore,0.038552,0.129567,0.031463
SVD,0.229744,0.132904,0.014061


In [23]:
bsl_options = {    
    'n_epochs': 5,
    'reg_u': 12,
    'reg_i': 5
}
algo = surprise.BaselineOnly(bsl_options)
cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


{'fit_time': (0.004280805587768555,
  0.004551410675048828,
  0.004909038543701172),
 'test_rmse': array([0.10838131, 0.10669   , 0.10732521]),
 'test_time': (0.010094642639160156,
  0.008274316787719727,
  0.008963823318481445)}

In [24]:
from surprise.model_selection import train_test_split
from surprise import accuracy

In [25]:
trainset, testset = train_test_split(data, test_size=0.25)

In [63]:
len(data.raw_ratings)

48550

In [64]:
trainset.ur[trainset.to_inner_uid('한일인쇄')]

[(11829, 0.16666666666666666),
 (14462, 0.16666666666666666),
 (9627, 0.16666666666666666)]

In [85]:
len(testset)

12138

In [100]:
algo.verbose

True

In [67]:
from surprise.model_selection import train_test_split
from surprise import accuracy
trainset, testset = train_test_split(data, test_size=0.25)
algo = surprise.BaselineOnly(bsl_options)
predictions = algo.fit(trainset).test(testset)
accuracy.rmse(predictions)

Estimating biases using als...
RMSE: 0.0948


0.09479121847966467

In [158]:
predictions

[Prediction(uid='대원화성(주)', iid='강남화성(주)', r_ui=0.25, est=0.0713546249787615, details={'was_impossible': False}),
 Prediction(uid='레이트론(주)', iid='(주)한강옵토', r_ui=0.06, est=0.07664407070855517, details={'was_impossible': False}),
 Prediction(uid='(주)알엔지', iid='주식회사 현대메디팜', r_ui=0.0007587253414264037, est=0.023271141663821826, details={'was_impossible': False}),
 Prediction(uid='인테르모다(주)', iid='썬텍스', r_ui=0.18181818181818182, est=0.08474144754431556, details={'was_impossible': False}),
 Prediction(uid='주식회사 케이지무역', iid='바인무역', r_ui=0.1111111111111111, est=0.08288390843425794, details={'was_impossible': False}),
 Prediction(uid='(주)한울트래이딩', iid='해마루 주식회사', r_ui=0.038461538461538464, est=0.05118428527279685, details={'was_impossible': False}),
 Prediction(uid='주식회사 다솜물산', iid='동양섬유(주)', r_ui=0.0425531914893617, est=0.09052429310521917, details={'was_impossible': False}),
 Prediction(uid='M&SDIEENGINEERING.INC.', iid='유진정밀', r_ui=0.09090909090909091, est=0.09654894957913572, details={'was_imp

In [70]:
data.raw_ratings

[('(사)경남농수산식품수출협회', '(주)에코맘의산골이유식 농업회사법인', 0.35714285714285715, None),
 ('(사)경남농수산식품수출협회', '국대씨푸드', 0.07142857142857142, None),
 ('(사)경남농수산식품수출협회', '농업회사법인(주)강림오가닉', 0.07142857142857142, None),
 ('(사)경남농수산식품수출협회', '슬로푸드 주식회사 농업회사법인', 0.14285714285714285, None),
 ('(사)경남농수산식품수출협회', '아로니아상사 영농조합법인', 0.07142857142857142, None),
 ('(사)경남농수산식품수출협회', '우포바이오푸드영농조합법인', 0.03571428571428571, None),
 ('(사)경남농수산식품수출협회', '재단법인 남해마늘연구소', 0.03571428571428571, None),
 ('(사)경남농수산식품수출협회', '청양식품영농조합법인', 0.21428571428571427, None),
 ('(유)브이피에이치아이', '(주)디와이시', 0.4142212189616253, None),
 ('(유)브이피에이치아이', '(주)오토탑', 0.04853273137697517, None),
 ('(유)브이피에이치아이', '(주)위코(WECO)', 0.3115124153498871, None),
 ('(유)브이피에이치아이', '(주)코덱스', 0.10270880361173815, None),
 ('(유)브이피에이치아이', '(주)평화발레오', 0.003386004514672686, None),
 ('(유)브이피에이치아이', '(주)홍성브레이크', 0.01580135440180587, None),
 ('(유)브이피에이치아이', '발레오전장시스템즈코리아(주)', 0.023702031602708805, None),
 ('(유)브이피에이치아이', '와이엠텍 주식회사', 0.01580135440180587, None),
 ('(유)브이피에이치아이', '정

In [89]:
algo.estimate(1, 1000)

0.05356295760991309

In [125]:
trainset.to_raw_iid(0)

'오성텍'

In [128]:
trainset.to_raw_uid(0)

'purisys'

In [135]:
trainset.to_inner_iid('(주)효성구미 1공장')

11725

In [134]:
trainset.to_inner_uid('(주) 동 승')

2854

In [106]:
df.loc[df.byr == 'purisys']

,byr,spl,div_cnt
21570,purisys,(주)필텍,0.06250
21571,purisys,능대,0.06250
21572,purisys,디에스피앤비,0.12500
21573,purisys,삼본엔프러스,0.06250
21574,purisys,앨트웰텍(주),0.06250
21575,purisys,에이피에스테크 주식회사,0.03125
21576,purisys,오성텍,0.12500
21577,purisys,우성아쿠아,0.03125
21578,purisys,제이앤비(J&B),0.03125
21579,purisys,주식회사 에이치비테크,0.09375


In [157]:
algo.estimate(2854, 3)

0.05811735019116342

In [162]:
def get_Iu(uid):
    """Return the number of items rated by given user
    
    Args:
        uid: The raw id of the user.
    Returns:
        The number of items rated by the user.
    """
    
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError:  # user was not part of the trainset
        return 0
    
def get_Ui(iid):
    """Return the number of users that have rated given item
    
    Args:
        iid: The raw id of the item.
    Returns:
        The number of users that have rated the item.
    """
    
    try:
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:  # item was not part of the trainset
        return 0

rcm_df = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])    
rcm_df['Iu'] = rcm_df.uid.apply(get_Iu)
rcm_df['Ui'] = rcm_df.iid.apply(get_Ui)
rcm_df['err'] = abs(rcm_df.est - rcm_df.rui)

In [ ]:
rcm_df.sort_values(by='err').head(20)

In [164]:
rcm_df.sort_values(by='err').tail(10)

,uid,iid,rui,est,details,Iu,Ui,err
1905,주식회사 제이 앤 씨,(주)우성하이텍,0.845070,0.072446,{'was_impossible': False},2,1,0.772624
9692,(유)포리코리아,(주)한국훼스토,0.830769,0.050865,{'was_impossible': False},9,0,0.779904
2131,(주)신성산업,(주) 엠플랜,0.851064,0.055282,{'was_impossible': False},7,0,0.795782
4690,(주)케이엠글로벌,(주)안인터내셔날,0.859375,0.053475,{'was_impossible': False},8,0,0.805900
11860,주식회사씨에스오토파트,(주)한라홀딩스,0.910798,0.092941,{'was_impossible': False},4,1,0.817857
5284,(주)그랑에스피오,주식회사 동방메디컬,0.877778,0.055031,{'was_impossible': False},4,1,0.822747
4035,주식회사 무진,(주)동그랑,0.904762,0.070090,{'was_impossible': False},1,0,0.834672
1694,하이트진로(주),주식회사 진로소주,0.877527,0.035145,{'was_impossible': False},12,0,0.842382
1551,주식회사다우무역,에이치엔에스하이텍(주),0.949477,0.060823,{'was_impossible': False},2,4,0.888655
2757,주식회사 카파야트인터내셔널,헤비파츠 (주),0.981221,0.058527,{'was_impossible': False},4,0,0.922694


In [ ]:
from collections import Counter

import matplotlib.pyplot as plt
import matplotlib
%matplotlib notebook
matplotlib.style.use('ggplot')

counter = Counter([r for (_, r) in trainset.ir[trainset.to_inner_iid('(주)현대화섬')]])
pd.DataFrame.from_dict(counter, orient='index').plot(kind='bar', legend=False)
plt.xlabel('Rating value')
plt.ylabel('Number of users')
plt.title('Number of users having rated item (주)현대화섬')

### TEST Rating before drop few ratings (trd_cnt)

In [ ]:
df.groupby('trd_cnt').agg('size').sum()

In [ ]:
df.groupby('trd_cnt').agg('size').loc[701:].sum()

In [ ]:
import sys
rating = {1:[1,1], 2:[2,2], 3:[3,4], 4:[5,7], 5:[8,12], 
          6:[13,23], 7:[24,50], 8:[51,160], 9:[161,700], 10:[701,sys.maxsize]}

In [ ]:
21477 + 13957 + 10439 + 6577 + 5284 + 4303 + 3603 + 2929 + 1634 + 589

In [ ]:
for i in rating:
    df.loc[(df.trd_cnt >= rating[i][0]) & (df.trd_cnt <= rating[i][1]), 'trd_cnt'] = i    

In [ ]:
df.head()

In [ ]:
df.groupby('trd_cnt').agg('size').plot.bar()

In [ ]:
df.head()

In [ ]:
df.groupby('byr').agg('size').sort_values(ascending=False)[:10].plot()

### Make Surprise Data from Dataframe

In [ ]:
reader = Reader(rating_scale=(1,10))

In [ ]:
data = Dataset.load_from_df(df, reader)

In [ ]:
from surprise.model_selection import cross_validate
bsl_options = {
    'method': 'als',
    'n_epochs': 5,
    'reg_u': 12,
    'reg_i': 5
}
algo = surprise.BaselineOnly(bsl_options)
cross_validate(algo, data)

## By AMT

In [ ]:
trade_df[['BYRORGNM1', 'SPLYORGNM1', 'AMT', 'CUR']].head()

In [ ]:
trade_df = trade_df[['BYRORGNM1', 'SPLYORGNM1', 'AMT', 'CUR']]

In [ ]:
trade_df.CUR.drop_duplicates()

In [ ]:
trade_df = trade_df.loc[(trade_df.CUR == "USD") | (trade_df.CUR == "KRW")]

In [ ]:
trade_df.loc[trade_df.CUR == 'USD'].head()

In [ ]:
trade_df.loc[trade_df.CUR == 'KRW'].head()

In [ ]:
def calculate_krw(amt: float, cur: str) -> float:
    if cur == 'KRW':
        return amt
    elif cur == 'USD':
        return amt * 1244

In [ ]:
trade_df['AMT_KRW'] = trade_df.apply(lambda x: calculate_krw(x['AMT'], x['CUR']), axis=1)

In [ ]:
trade_df.loc[[0, 35800]]

In [ ]:
df = trade_df[['SPLYORGNM1', 'BYRORGNM1', 'AMT_KRW']]

In [ ]:
df.head()

In [ ]:
df = df.groupby(['SPLYORGNM1', 'BYRORGNM1']).agg('sum').sort_values('AMT_KRW', ascending=False).reset_index()

In [ ]:
df.head()

In [ ]:
df.columns = ['spl', 'byr', 'krw']

In [ ]:
df.shape

In [ ]:
df_table = df.set_index(["spl", "byr"]).unstack()
df_table.shape

In [ ]:
plt.imshow(df_table)
plt.grid(False)
plt.xlabel("byr")
plt.ylabel("spl")
plt.title("Trade Matrix")
plt.show()

In [ ]:
list(df_table.index).index('POSCO')

In [ ]:
val = '주식회사 포스코대우'
[(index, row.index(val)) for index, row in enumerate(list(df_table.columns)) if val in row]

In [ ]:
df_table.iloc[13660:13670, 13740:13750]

In [ ]:
df_table.count(axis=1).sort_values(ascending=False)[:100].plot.box

In [ ]:
plt.imshow(df_table.loc[df_table.count(axis=1).sort_values(ascending=False)[:100].index])
plt.grid(False)
plt.xlabel("byr")
plt.ylabel("spl")
plt.title("Trade Matrix")
plt.show()

In [ ]:
plt.imshow(df_table.iloc[13660:13670, 13740:13750])
plt.grid(False)
plt.xlabel("byr")
plt.ylabel("spl")
plt.title("Trade Matrix")
plt.show()

In [ ]:
df_table.count(axis=0).sort_values(ascending=False)

## Make dataframe to surprise dataset

In [ ]:
df.krw.max()

In [ ]:
df.krw.min()

In [ ]:
from surprise import Reader
from surprise import Dataset

In [ ]:
reader = Reader(rating_scale)

In [ ]:
df.head()

In [ ]:
df.krw.max()

In [ ]:
reader=Reader(rating_scale=(0, 5))

In [ ]:
df

In [ ]:
df.krw = df.krw / df.krw.max() * 5.0

In [ ]:
data = Dataset.load_from_df(df[['spl', 'byr', 'krw']], reader=reader)

In [ ]:
data.raw_ratings

In [ ]:
from surprise.model_selection import KFold

bsl_options = {
    'method': 'als',
    'n_epochs': 500,
    'reg_u': 12,
    'reg_i': 5
}
algo = surprise.BaselineOnly(bsl_options)

np.random.seed(0)
acc = np.zeros(3)
cv = KFold(3)
for i, (trainset, testset) in enumerate(cv.split(data)):
    algo.fit(trainset)
    predictions = algo.test(testset)
    acc[i] = surprise.accuracy.rmse(predictions, verbose=True)
acc.mean()

In [ ]:
from surprise.model_selection import cross_validate

sim_options = {'name': 'msd'}
algo = surprise.KNNBasic(sim_options=sim_options)
cross_validate(algo, data)["test_mae"].mean()

In [ ]:
data_ml = surprise.Dataset.load_builtin('ml-100k')

In [ ]:
data.raw_ratings

In [ ]:
data_ml.raw_ratings

# Trade count

In [ ]:
df = trade_df[['BYRORGNM1', 'SPLYORGNM1']]

In [ ]:
df = df.groupby(['BYRORGNM1', 'SPLYORGNM1']).agg('size').reset_index()

In [ ]:
df.head()

In [ ]:
df.columns = ['byr', 'spl', 'trd_cnt']

In [ ]:
df.head()

In [ ]:
df.sort_values('trd_cnt').trd_cnt.reset_index(drop=True).plot()

In [ ]:
len(df.loc[df.trd_cnt < 3])

In [ ]:
len(df.loc[df.trd_cnt <= 14])

In [ ]:
df.sort_values('trd_cnt').trd_cnt.reset_index(drop=True)[35434:58914].plot.box()

In [ ]:
df.sort_values('trd_cnt').trd_cnt.reset_index(drop=True)[35434:58914].plot()

In [ ]:
df = df.sort_values('trd_cnt')[35434:58914].reset_index(drop=True)

In [ ]:
df.tail()

In [ ]:
df.groupby('spl').agg('size').sort_values().plot()

In [ ]:
df.groupby('spl').agg('size').sort_values().tail()

In [ ]:
df.groupby('spl').agg('size').plot.box()

In [ ]:
# refer: https://nbviewer.jupyter.org/github/NicolasHug/Surprise/blob/master/examples/notebooks/KNNBasic_analysis.ipynb